# Performance Comparison

In [ ]:
# Agent config
num_agents = 100
agent_radius = 0.1  

# Map config
map_type = "obstacle" # Available maps: empty, corridor, obstacle, cross
num_samples = 500    # Available num_samples: 100, 200, 300, 400, 500

# Instance config, ratio of starts and goals

starts_weight = [0.5, 0.5]
goals_weight = [0.2, 0.8]


In [ ]:
# Load map and instance
import os
import pickle

from swarm_prm.utils.agent_assignment import get_agent_assignment

map_fname = "{}_{}.pkl".format(map_type, num_samples)
fname = os.path.join("../maps", map_fname)
with open(fname, "rb") as f:
    gaussian_prm = pickle.load(f)

starts_agent_count = get_agent_assignment(num_agents, starts_weight)
goals_agent_count = get_agent_assignment(num_agents, goals_weight)

print(starts_agent_count, goals_agent_count)

In [ ]:
from swarm_prm.solvers.macro import SOLVER_REGISTRY

# solver_cls = SOLVER_REGISTRY["FormationControlSolver"]
solver_cls = SOLVER_REGISTRY["TEGSolver"]

solver = solver_cls(gaussian_prm, agent_radius, 
              starts_agent_count=starts_agent_count, goals_agent_count=goals_agent_count,
               num_agents=num_agents, time_limit=10)
solution = solver.solve()
assert solution["success"], "solver failed."
timestep = solution["timestep"]
paths = solution["paths"]
g_nodes = solution["g_nodes"]
starts_idx = solution["starts_idx"]
goals_idx = solution["goals_idx"]

In [ ]:
from matplotlib import pyplot as plt
from swarm_prm.utils import paths_to_macro
from swarm_prm.solvers.micro import EvaluationSolver

macro_solution = paths_to_macro(paths)

timestep = max(macro_solution.keys())

solver = EvaluationSolver(g_nodes, macro_solution, agent_radius, timestep, 
                                     num_agents, 
                                     starts_idx,
                                     goals_idx,
                                     starts_agent_count,
                                     goals_agent_count
                                     )

single_agent_paths, cost = solver.solve()
fig, ax = gaussian_prm.visualize_map()

cmap = plt.get_cmap("rainbow")
colors = [cmap(i / num_agents) for i in range(num_agents)]

for i, path in enumerate(single_agent_paths):
    x_coords = [loc[0] for loc in path]
    y_coords = [loc[1] for loc in path]
    ax.plot(x_coords, y_coords, '-', label='Path', color=colors[i], linewidth=0.8, alpha=0.8)

plt.show()

In [ ]:
# Simulate with APF
from swarm_prm.solvers.micro.apf import APFWaypointSolver

apf_solver = APFWaypointSolver(gaussian_prm.raw_map, single_agent_paths, agent_radius,
                                attract_coeff=1,
                                agent_repel_coeff=0.2,
                                repel_coeff=0.5,
                                obs_thresh=1,
                                step_size=0.5,
                                reach_dist=5,
                                max_timestep_iter=1000
                               )
paths = apf_solver.get_solution()

In [ ]:
fig, ax = gaussian_prm.visualize_map()
for i, path in enumerate(paths):
    x_coords = [loc[0] for loc in path]
    y_coords = [loc[1] for loc in path]
    ax.plot(x_coords, y_coords, '-', label='Path', color=colors[i], linewidth=0.8, alpha=0.8)

plt.show()

In [ ]:
# Animate path
from IPython.display import Image

fig_path = "solutions"
fig, ax = gaussian_prm.visualize_map()
apf_solver.animate_solution(paths, fig, ax, fig_path=fig_path)
Image(filename="solutions/apf_solution.gif")

